<a href="https://colab.research.google.com/github/SeneKim/TAVE/blob/main/11th/SamsungCrawling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 네이버 API 사용

In [ ]:
import os
import sys
import urllib.request
import datetime
import time
import json

In [ ]:
client_id = '7mgOzIKx3rx3UlqONPVy'
client_secret = '_rGczNnaLl'

In [ ]:
#[CODE 1]
def getRequestUrl(url):
    req = urllib.request.Request(url)
    req.add_header("X-Naver-Client-Id", client_id)
    req.add_header("X-Naver-Client-Secret", client_secret)

    try:
        response = urllib.request.urlopen(req)
        if response.getcode() == 200:
            print("[%s]Url Request Success" % datetime.datetime.now())
            return response.read().decode('utf-8')
    except Exception as e :
        print(e)
        print("[%s] Error for URL : %s" % (datetime.datetime.now(), url))
        return None

#[CODE 2]
def getNaverSearch(node, srcText, start, display):
    base = "https://openapi.naver.com/v1/search"
    node = "/%s.json" % node
    parameters = "?query=%s&start=%s&display=%s" % (urllib.parse.quote(srcText), start, display)

    url = base + node + parameters
    responseDecode = getRequestUrl(url)     #[CODE 1]

    if(responseDecode == None):
        return None
    else:
        return json.loads(responseDecode)

#[CODE 3]
def getPostData(post, jsonResult, cnt):
    title = post['title']
    description = post['description']
    org_link = post['originallink']
    link = post['link']

    pDate = datetime.datetime.strptime(post['pubDate'], '%a, %d %b %Y %H:%M:%S +0900')
    pDate = pDate.strftime('%Y-%m-%d %H:%M:%S')

    jsonResult.append({'cnt':cnt, 'title':title, 'description':description,
                       'org_link':org_link, 'link':org_link, 'pDate':pDate})
    return

#[CODE 0]
def main():
    node = 'news'  #크롤링한 대상
    srcText = input('검색어를 입력하세요: ')
    cnt = 0
    jsonResult = []

    jsonResponse = getNaverSearch(node, srcText, 1, 100)  #[CODE 2]
    total = jsonResponse['total']

    while((jsonResponse != None) and (jsonResponse['display'] != 0)):
        for post in jsonResponse['items']:
            cnt+= 1
            getPostData(post, jsonResult, cnt)   #[CODE 3]

        start = jsonResponse['start'] + jsonResponse['display']
        jsonResponse = getNaverSearch(node, srcText, start, 100)   #[CODE 2]

    print('전체 검색 : %d 건' %total)

    with open('%s_naver_%s.json' % (srcText, node), 'w', encoding='utf8') as outfile:
        jsonFile = json.dumps(jsonResult, indent=4, sort_keys = True, ensure_ascii = False)

        outfile.write(jsonFile)

    print("가져온 데이터 : %d 건" %(cnt))
    print('%s_naver_%s.json SAVED' % (srcText, node))'''


'   jsonResponse = getNaverSearch(node, srcText, 1, 100)  #[CODE 2]\n    total = jsonResponse[\'total\']\n    \n    while((jsonResponse != None) and (jsonResponse[\'display\'] != 0)):\n        for post in jsonResponse[\'items\']:\n            cnt+= 1\n            getPostData(post, jsonResult, cnt)   #[CODE 3]\n            \n        start = jsonResponse[\'start\'] + jsonResponse[\'display\']\n        jsonResponse = getNaverSearch(node, srcText, start, 100)   #[CODE 2]\n        \n    print(\'전체 검색 : %d 건\' %total)\n        \n    with open(\'%s_naver_%s.json\' % (srcText, node), \'w\', encoding=\'utf8\') as outfile:\n        jsonFile = json.dumps(jsonResult, indent=4, sort_keys = True, ensure_ascii = False)\n            \n        outfile.write(jsonFile)\n            \n    print("가져온 데이터 : %d 건" %(cnt))\n    print(\'%s_naver_%s.json SAVED\' % (srcText, node))'

In [ ]:
if __name__ == '__main__':
    main()

## 동적 크롤링

In [ ]:
!pip install webdriver_manager
!pip install selenium

In [ ]:
#크롤링시 필요한 라이브러리 불러오기
from bs4 import BeautifulSoup
import requests
import re
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options  import Options

In [ ]:
# 웹 드라이버 설정 - 웹 자동화 작업 수행
options.add_experimental_option("excludeSwitches", ["enable-automation"]) #특정 옵션 추가
options.add_experimental_option("useAutomationExtension", False)

In [ ]:
def makePgNum(num):
    if num == 1:
        return num
    elif num == 0:
        return num+1
    else:
        return num + 9 * (num-1)

In [ ]:
# 크롤링할 url 생성하는 함수 만들기(검색어, 크롤링 시작 페이지, 크롤링 종료 페이지)
def makeUrl(search,start_pg,end_pg):
    if start_pg == end_pg:
        start_page = makePgNum(start_pg)
        url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(start_page)
        print("생성url: ",url)
        return url
    else:
        urls= []
        for i in range(start_pg,end_pg+1):
            page = makePgNum(i)
            url = "https://search.naver.com/search.naver?where=news&sm=tab_pge&query=" + search + "&start=" + str(page)
            urls.append(url)
        #print("생성url: ",urls)
        return urls

In [ ]:
#검색어 입력
search = input("검색할 키워드를 입력해주세요:")

검색할 키워드를 입력해주세요:삼성전자 주가


In [ ]:
#검색 시작할 페이지 입력
page = int(input("\n크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 시작 페이지: ",page,"페이지")
#검색 종료할 페이지 입력
page2 = int(input("\n크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):")) # ex)1 =1페이지,2=2페이지...
print("\n크롤링할 종료 페이지: ",page2,"페이지")

# naver url 생성

search_urls = makeUrl(search, page, page2)


크롤링할 시작 페이지를 입력해주세요. ex)1(숫자만입력):1

크롤링할 시작 페이지:  1 페이지

크롤링할 종료 페이지를 입력해주세요. ex)1(숫자만입력):40

크롤링할 종료 페이지:  40 페이지
생성url:  ['https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자 주가&start=1', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자 주가&start=11', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자 주가&start=21', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자 주가&start=31', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자 주가&start=41', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자 주가&start=51', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자 주가&start=61', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자 주가&start=71', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자 주가&start=81', 'https://search.naver.com/search.naver?where=news&sm=tab_pge&query=삼성전자 주가&start=91', 'https://search.

In [ ]:
# 버전에 상관 없이 os에 설치된 크롬 브라우저 사용
service=Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
driver.implicitly_wait(3)
driver.add_argument("headless")

naver_urls = []

for i in search_urls:
    driver.get(i)
    time.sleep(1)  # 대기시간 변경 가능

    # 네이버 기사 눌러서 제목 및 본문 가져오기#
    # 네이버 기사가 있는 기사 css selector 모아오기
    a = driver.find_elements(By.CSS_SELECTOR, 'a.info')

    # 위에서 생성한 css selector list 하나씩 클릭하여 본문 url얻기
    for i in a:
        i.click()

        # 현재탭에 접근
        driver.switch_to.window(driver.window_handles[1])
        time.sleep(3)  # 대기시간 변경 가능

        # 네이버 뉴스 url만 가져오기
        url = driver.current_url
        #print(url)

        if "news.naver.com" in url:
            naver_urls.append(url)

        else:
            pass
        # 현재 탭 닫기
        driver.close()
        # 다시처음 탭으로 돌아가기(매우 중요!!!)
        driver.switch_to.window(driver.window_handles[0])

print(naver_urls)

###naver 기사 본문 및 제목 가져오기###

# ConnectionError방지
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) Chrome/98.0.4758.102"}

titles = []
contents = []
for i in naver_urls:
    original_html = requests.get(i, headers=headers)
    html = BeautifulSoup(original_html.text, "html.parser")
    # 검색결과확인시
    # print(html)

    # 뉴스 제목 가져오기
    title = html.select("div#ct > div.media_end_head.go_trans > div.media_end_head_title > h2")
    # list합치기
    title = ''.join(str(title))
    # html태그제거
    pattern1 = '<[^>]*>'
    title = re.sub(pattern=pattern1, repl='', string=title)
    titles.append(title)

    # 뉴스 본문 가져오기
    content = html.select("div#dic_area")

    # 기사 텍스트만 가져오기
    # list합치기
    content = ''.join(str(content))

    # html태그제거 및 텍스트 다듬기
    content = re.sub(pattern=pattern1, repl='', string=content)
    pattern2 = """[\n\n\n\n\n// flash 오류를 우회하기 위한 함수 추가\nfunction _flash_removeCallback() {}"""
    content = content.replace(pattern2, '')

    contents.append(content)

print(titles)
print(contents)

# 데이터프레임으로 정리(titles,url,contents)
import pandas as pd

news_df = pd.DataFrame({'title': titles, 'link': naver_urls, 'content': contents})

news_df.to_csv('NaverNews_%s.csv' % search, index=False, encoding='utf-8')

WebDriverException: ignored

## 기타

기간: 2022.01 ~ 2023.05.31 → 크롤링 / 네이버에서 실행

호재 뉴스 키워드

- 주가 상승
- 고배당
- 실적 개선
- 수주 확보

악재 뉴스 키워드

- 주가 하락
- 실적 약화
- 저배당
- 제품 결함 / 리콜
- 시장 악화


In [ ]:
# 반드시 설치 필요
!pip install newspaper3K

In [ ]:
import requests
import csv
from bs4 import BeautifulSoup
from newspaper import Article, Config
from tqdm import tqdm
from datetime import datetime

# newspaper 설정
config = Config()
config.language = 'ko'

In [ ]:
'''
검색하기 위한 url을 만드는 함수
query = "검색어 입력"
ds = "시작 날짜"
de = "종료 날짜"

날짜 형식은 반드시 2023.06.24 같이 "."을 구분자로 사용할 것

'''

def make_dir(keyword, start_date, end_date):
  keyword = keyword.replace(" ", "+")
  conv_start_date = start_date.replace(".", "")
  conv_end_date = end_date.replace(".", "")
  base_url = f"https://search.naver.com/search.naver?where=news&sm=tab_pge&query={keyword}&sort=0&photo=0&field=0&pd=3&ds={start_date}&de={end_date}&cluster_rank=174&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so:r,p:from{conv_start_date}to{conv_end_date},a:all" #키워드 입력

  return base_url

In [ ]:
# 검색할 키워드와 날짜를 지정합니다. 시장 악화 /제품 결함 - 리콜
keyword = "삼성전자 리콜"
start_date = "2022.01.01"
end_date = "2023.05.31"

base_url = make_dir(keyword, start_date, end_date)

In [ ]:
# page_num은 건드리지 말 것
page_num = 1

# 크롤링 정보를 저장할 리스트 생성
articles = []

while True:
  # URL에 GET 요청을 보내고 응답을 받음, base에 기반한 page url 생성
  url = base_url + f"&start={page_num}"
  response = requests.get(url) # -> 날짜가 검색된 네이버 뉴스 페이지로 이동

  if response.status_code == 200: # 응답이 있음
    soup = BeautifulSoup(response.text, 'html.parser')

    # 기사 리스트를 찾아옴
    article_list = soup.select('ul.list_news > li')

    if len(article_list) == 0:
        break

    # 리스트에 기반하여
    for article in tqdm(article_list):
        # 기사 제목 추출
        title = article.select_one('a.news_tit').text

        # 기사 링크 추출 - 하나하나의 링크 추출
        article_url = article.select_one('a.news_tit')['href']

        try:
          # newspaper 객체를 통해 article 접속 후 데이터 추출
          article_obj = Article(article_url, config=config)
          article_obj.download()  # 기사 내용 다운로드
          article_obj.parse()  # 기사 파싱

          articles.append({
            '제목': article_obj.title,
            '날짜': article_obj.publish_date,
            '본문': article_obj.text
                })

        except:
          continue

  # start_num은 넘어가면서 10씩 증가
  page_num += 10

# CSV 파일로 저장
csv_file = f'{keyword}_To_{start_date}_From_{end_date}.csv'
fieldnames = ['제목', '날짜', '본문']

with open(csv_file, 'w', encoding='utf-8-sig', newline='') as f:
    writer = csv.DictWriter(f, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(articles)

print(f"\n뉴스 기사가 '{csv_file}' 파일로 저장되었습니다.")

100%|██████████| 7/7 [00:16<00:00,  2.30s/it]



뉴스 기사가 '삼성전자 리콜_To_2022.01.01_From_2023.05.31.csv' 파일로 저장되었습니다.
